In [1]:
import numpy as np
import random
from time import perf_counter as ttt

# allow func library to be imported
import sys
sys.path.append('../')

from func.convert import *

from dtype import Board, Move
import get
import indicator

# Reusable Functions and variables

In [2]:
valid_cells = [21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98]
var_folder = 'C:/Users/lenovo/Desktop/Coding/VSC Projects/Ches_Dev/ChessAI_3/vars/'

In [3]:
def display_stats(time_log, num_runs):
    print(f'Number of runs for all functions: {num_runs}')
    for func_name, dur_hist in time_log.items():
        if func_name != 'dummy':
            # function name
            print(f'{func_name.upper()}')
            # total time and average time
            total_time = sum(dur_hist)
            print(f'Total time: {total_time}s\nAverage time per run: {total_time/num_runs/1000}ms')
            # median time
            ord_hist = sorted(dur_hist)  # sort the duration log in ascending order
            if len(ord_hist)%2 == 0:
                median = (ord_hist[int(len(ord_hist)/2)] + ord_hist[int(len(ord_hist)/2-1)])/2
            else:
                median = ord_hist[int(len(ord_hist)/2-0.5)]
            print(f'Median Time taken: {median}s')

In [4]:
def random_board_full_dict():
    piece_nums = {1:8, 2:2, 3:2, 4:2, 5:1, 6:1}  # number of pieces per side
    board = {}
    for side in [1, -1]:
        for piece_index, piece_num in piece_nums.items():
            # define possible cells for the piece to be in
            if piece_index == 1:
                valid_cells = [31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88]
            else:
                valid_cells = [21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98]
            
            # king cannot be placed next to another king. remove the area around the existing king from valid cells
            if piece_index*side == -6:
                for dir_to_surrounding in [-11, -10, -9, -1, 0, 1, 9, 10, 11]:  # loop through every cell around the king position
                    drop_cell = king_cell + dir_to_surrounding
                    try:
                        valid_cells.remove(drop_cell)
                    except ValueError:
                        pass

            for _ in range(piece_num):
                while True:
                    # select a random cell
                    rand_index = random.randint(0, len(valid_cells)-1)
                    cell = valid_cells[rand_index]

                    if cell not in board:
                        board[cell] = piece_index * side
                        break

            # record the white king position
            if piece_index == 6:
                king_cell = cell

    return board

In [5]:
def random_board_full_arr():
    return dict_to_arr(random_board_full_dict())

In [6]:
def random_valid_cell():
    index = random.randint(0, 63)
    return valid_cells[index]

In [7]:
def dummy(*args, **kwargs):
    pass

# Functions To Test

## dict_to_arr()

In [4]:
def dict_to_arr(dict) -> np.ndarray:
    output = np.zeros((8, 8), dtype=int)
    for cell, index in dict.items():
        cell = cell-21
        row = cell//10
        col = cell%10
        output[row, col] = index
    return output

dict_to_arr(ini_board)

array([[-4, -2, -3, -5, -6, -3, -2, -4],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 4,  2,  3,  5,  6,  3,  2,  4]])

## random_board_full_arr()

In [20]:
random_board_full_arr()

array([[ 2,  0,  4,  0,  0,  6,  0, -4],
       [ 2,  0, -1, -3,  0,  1,  0,  3],
       [ 1,  0,  1,  0,  1,  4,  0,  1],
       [-6, -1, -2,  0,  0,  0,  0,  0],
       [ 0, -3,  5,  0,  0,  0, -1, -1],
       [ 0,  1, -1,  3, -1,  0,  0,  0],
       [ 1, -1, -5,  0, -4, -1,  1,  0],
       [ 0, -2,  0,  0,  0,  0,  0,  0]])

## find_king()

In [7]:
'''
Function Definitions
'''

# using array representation of board
def find_king1(arr, index):
    coor = np.where(arr == index)
    return int(21 + 10*coor[0] + coor[1])


# using dict representation of board
def find_king2(dict_, index):
    for cell, piece_index in dict_.items():
        if piece_index == index:
            return cell


'''
Testing Hyperparameters
'''

# the first function to run seems to have inflated time to run
functions = [dummy, find_king1, find_king2]
king_index = [6, -6]
n_iter = 100000

total_runs = n_iter*2

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random board
    board_dict = random_board_full_dict()
    board_arr = dict_to_arr(board_dict)

    # loop through all functions
    for func_name, func in func_dict.items():
        # adjust input to function based on the function to be used
        if func_name == 'find_king1':
            inp = board_arr
        else:
            inp = board_dict
        # time the run
        start = ttt()
        for index in king_index:
            func(inp, index)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 200000
DUMMY
Total time: 0.18830760000069446s
Average time per run: 9.415380000034723e-10ms
Median Time taken: 1.1999999998124622e-06s
FIND_KING1
Total time: 2.9600272999991653s
Average time per run: 1.4800136499995827e-08ms
Median Time taken: 1.6800000004479898e-05s
FIND_KING2
Total time: 0.7530881999999188s
Average time per run: 3.765440999999594e-09ms
Median Time taken: 4.300000000512227e-06s


## Move Representaion: Custom DataType vs Tuple

In [12]:
'''
Utility Imports, Functions and Classes
'''

class TestData:
    
    def __init__(self, start, final, castle, promo):
        self.start = start
        self.final = final
        self.castle = castle
        self.promo = promo

def gen_random_data():
    r = random.randint
    return valid_cells[r(0, len(valid_cells)-1)], valid_cells[r(0, len(valid_cells)-1)], r(0, 10), r(2, 5)

'''
Test Functions
'''

def get_class(inp_tuple, inp_class):
    return inp_class.start, inp_class.final, inp_class.castle, inp_class.promo


def get_tuple(inp_tuple, inp_class):
    return inp_tuple[0], inp_tuple[1], inp_tuple[2], inp_tuple[3]

'''
Testing Hyperparameters
'''

# the first function to run seems to have inflated time to run
functions = [dummy, get_class, get_tuple]
king_index = [6, -6]
n_iter = 100000

total_runs = n_iter*2

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    start_cell, final_cell, castle, promo = gen_random_data()

    # loop through all functions
    for func_name, func in func_dict.items():
        # adjust input to function based on the function to be used
        inp_class = TestData(start_cell, final_cell, castle, promo)
        inp_tuple = (start_cell, final_cell, castle, promo)
        # time the run
        start = ttt()
        for index in king_index:
            func(inp_tuple, inp_class)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)


Number of runs for all functions: 200000
DUMMY
Total time: 0.08337610001058238s
Average time per run: 4.168805000529119e-10ms
Median Time taken: 7.000000152856956e-07s
GET_CLASS
Total time: 0.10065270000211513s
Average time per run: 5.032635000105756e-10ms
Median Time taken: 9.000000034120603e-07s
GET_TUPLE
Total time: 0.10080780000700429s
Average time per run: 5.040390000350214e-10ms
Median Time taken: 8.000000093488779e-07s


## pawn prelegal

In [20]:
'''
Test Functions
'''

def test1(board, cell, side, dummy_var=None):
    # 2 if statements
    new_cell = cell + 10*-side
    if get.side(board, new_cell) == 0:
        jump_cell = new_cell + 10*-side
        if get.side(board, jump_cell) == 0:
            jump_allowed = cell//10==8 if side == 1 else cell//10==3
            if jump_allowed:
                yield jump_cell
        yield new_cell

def test2(board, cell, side, dummy_var=None):
    # and
    new_cell = cell + 10*-side
    if get.side(board, new_cell) == 0:
        jump_cell = new_cell + 10*-side
        jump_allowed = cell//10==8 if side == 1 else cell//10==3
        if get.side(board, jump_cell) == 0 and jump_allowed:
            yield jump_cell
        yield new_cell

def test3(board, cell, side, dummy_var=None):
    # +
    new_cell = cell + 10*-side
    if get.side(board, new_cell) == 0:
        jump_cell = new_cell + 10*-side
        jump_allowed = cell//10==8 if side == 1 else cell//10==3
        if (get.side(board, jump_cell) == 0 + jump_allowed) == 2:
            yield jump_cell
        yield new_cell

'''
Testing Hyperparameters
'''

functions = [dummy, test1, test2, test3]    # the first function to run seems to have inflated time to run
n_iter = 100000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    board = Board(random_board_full_arr(), None)
    cell = random_valid_cell()
    side = [1, -1][random.randint(0, 1)]

    # loop through all functions
    for func_name, func in func_dict.items():

        # time the run
        start = ttt()
        func(board, cell, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

# go with test2 for readability

Number of runs for all functions: 100000
DUMMY
Total time: 0.10424290001037662s
Average time per run: 1.042429000103766e-09ms
Median Time taken: 6.000000212225132e-07s
TEST1
Total time: 0.15607829999737532s
Average time per run: 1.5607829999737533e-09ms
Median Time taken: 7.999999525054591e-07s
TEST2
Total time: 0.0817388000034498s
Average time per run: 8.17388000034498e-10ms
Median Time taken: 5.999999643790943e-07s
TEST3
Total time: 0.08004209999944578s
Average time per run: 8.004209999944578e-10ms
Median Time taken: 6.000000212225132e-07s


## find_king

In [11]:
'''
Utility Imports, Functions and Classes
'''

class TestData:
    
    def __init__(self, board_dict):
        self.dict = board_dict

def gen_random():
    return TestData(random_board_full_dict())


'''
Test Functions
'''

def find_king1(self):
    self.king_pos = {}
    for cell, index in self.dict.items():
        if abs(index) == 6:
            self.king_pos[index] = cell

def find_king2(self):
    self.king_pos = {
        -6: list(self.dict.keys())[list(self.dict.values()).index(-6)], 
        6: list(self.dict.keys())[list(self.dict.values()).index(6)]
    }


'''
Testing Hyperparameters
'''

functions = [dummy, find_king1, find_king2]    # the first function to run seems to have inflated time to run
n_iter = 200000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    board = gen_random()

    # loop through all functions
    for func_name, func in func_dict.items():

        # time the run
        start = ttt()
        func(board)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 200000
FIND_KING1
Total time: 1.8380470000171272s
Average time per run: 9.190235000085637e-09ms
Median Time taken: 5.500000042957254e-06s
FIND_KING2
Total time: 1.649494500000742s
Average time per run: 8.24747250000371e-09ms
Median Time taken: 5.000000101063051e-06s


## indicator.check()

In [36]:
'''
Utility Imports, Functions and Classes
'''

import get

class TestData:

    with open(var_folder + 'valid_cells.txt', 'r') as f:
        valid_cells = eval(f.read())

    with open(var_folder + 'threats.txt', 'r') as f:
        threats = eval(f.read())
    
    with open(var_folder + 'threats3.txt', 'r') as f:
        threats3 = eval(f.read())
    
    def __init__(self, board_dict):
        self.dict = board_dict
        self.find_king()

    def find_king(self):
        self.king_pos = {
            -1: list(self.dict.keys())[list(self.dict.values()).index(-6)], 
            1: list(self.dict.keys())[list(self.dict.values()).index(6)]
        }

def gen_random():
    return TestData(random_board_full_dict())



'''
Test Functions
'''

def check1(board, side):
    for n_times, threats_type in board.threats[side].items():
        # king position saved in Board
        for direction, indexes in threats_type.items():
            if get.first_in_direction(board, board.king_pos[side], direction, n_times) in indexes:
                return True
    return False

def check2(board, side):
    for lst in board.threats3[side]:
        for direction, indexes in lst:
            piece = get.index(board, board.king_pos[side]+direction)
            if piece in indexes:
                return True
            elif piece != 0:
                break
        
    return False


'''
Testing Hyperparameters
'''

functions = [dummy, check1, check2]    # the first function to run seems to have inflated time to run
n_iter = 200000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    board = gen_random()

    # loop through all functions
    for func_name, func in func_dict.items():

        for side in [-1, 1]:
        # time the run
            start = ttt()
            func(board, side)
            end = ttt()
            dur = end-start
            # log the time taken
            try:
                time_log[func_name].append(dur)
            except:
                time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 200000
CHECK1
Total time: 16.017831600011505s
Average time per run: 8.008915800005752e-08ms
Median Time taken: 3.349999997226405e-05s
CHECK2
Total time: 11.859241100145937s
Average time per run: 5.929620550072968e-08ms
Median Time taken: 2.4399999801971717e-05s


## sign

In [13]:
'''
Utility Imports, Functions and Classes
'''

def gen_random_data():
    index = random.randint(0, 2)
    if index == 0:
        return None
    else:
        return random.randint(-6, 6)

'''
Test Functions
'''

def sign1(num):
    if num == None:
        return 0
    elif num > 0:
        return 1
    elif num < 0:
        return -1
    return 0

def sign2(num):
    if num == None or num == 0:
        return 0
    elif num > 0:
        return 1
    return -1

def sign3(num):
    if num != None:
        if num > 0:
            return 1
        elif num < 0:
            return -1
    return 0

def sign4(num):
    if num != None:
        return np.sign(num)
    return 0

def sign5(num):
    try:
        return np.sign(num)
    except:
        return 0

def sign6(num):
    return np.sign(num)

'''
Testing Hyperparameters
'''

functions = [dummy, sign1, sign2, sign3, sign4, sign5, sign6, np.sign, dummy]    # the first function to run seems to have inflated time to run
n_iter = 200000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    num = gen_random_data()

    # loop through all functions
    for func_name, func in func_dict.items():

        if func_name == 'sign' or 'sign6':
            num = random.randint(-6, 6)

        # time the run
        start = ttt()
        func(num)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 200000
SIGN1
Total time: 0.10390620008661244s
Average time per run: 5.195310004330622e-10ms
Median Time taken: 4.0000008993956726e-07s
SIGN2
Total time: 0.11602129998891542s
Average time per run: 5.80106499944577e-10ms
Median Time taken: 4.999999418942025e-07s
SIGN3
Total time: 0.10018640005341695s
Average time per run: 5.009320002670847e-10ms
Median Time taken: 4.0000008993956726e-07s
SIGN4
Total time: 0.341882699968437s
Average time per run: 1.709413499842185e-09ms
Median Time taken: 1.2999998943996616e-06s
SIGN5
Total time: 0.29700560000446785s
Average time per run: 1.4850280000223394e-09ms
Median Time taken: 1.2000000424450263e-06s
SIGN6
Total time: 0.2810381999756828s
Average time per run: 1.405190999878414e-09ms
Median Time taken: 1.2000000424450263e-06s
SIGN
Total time: 0.23287790003109876s
Average time per run: 1.1643895001554936e-09ms
Median Time taken: 9.99999883788405e-07s


## legal

In [8]:
'''
Utility Imports, Functions and Classes
'''

from dtype import Move, Board
from random import randint as r

def gen_random_board_move():
    board = Board(random_board_full_dict())
    while True:
        random_move_data = gen_random_move_data()
        piece_sign = get.side(board, random_move_data[0])
        if piece_sign != 0:
            break
    
    move = Move(piece_sign, *random_move_data)
    return board, move

def gen_random_move_data():
    r = random.randint
    return valid_cells[r(0, len(valid_cells)-1)], valid_cells[r(0, len(valid_cells)-1)], 0, 0
    

'''
Test Functions
'''

def legal1(board, move):
    # pseudomove every move
    new_board = board.move_copy(move)
    return indicator.check(new_board, move.side)

def legal2(board, move):
    # if check: pseudomove
    # else: only pseudomove ally pieces which are in slider positions
    if board.check[move.side]:
        new_board = board.move_copy(move)
        return indicator.check(new_board, move.side)
    # elif check if piece is an ally in slider position from king

    king_to_piece = move.start - board.king_pos[move.side]
    for direction in [1, 9, 10, 11]:
        if king_to_piece % direction == 0:
            if king_to_piece // direction in list(range(-7, 8)):
                new_board = board.move_copy(move)
                return indicator.check(new_board, move.side)
    
    return True


def legal3(board, move):
    # if check: pseudomove
    # else: see if the 2nd piece behind ally piece in slider positions is a threat (whether the ally piece is pinned)
    if board.check[move.side]:
        new_board = board.move_copy(move)
        return indicator.check(new_board, move.side)

    king_to_piece = move.start - board.king_pos[move.side]
    for direction in [-11, -10, -9, -1, 1, 9, 10, 11]:
        if king_to_piece % direction == 0:
            if king_to_piece // direction in list(range(1, 8)):
                piece_behind = get.first_in_direction(board, move.start, direction, 6)
                if piece_behind == -move.side * board.slider_threats[direction]:
                    return False
    return True

'''
Testing Hyperparameters
'''

functions = [dummy, legal1, legal2, legal3]    # the first function to run seems to have inflated time to run
n_iter = 100000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data
    board, move = gen_random_board_move()

    # loop through all functions
    for func_name, func in func_dict.items():

        # time the run
        start = ttt()
        func(board, move)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 100000
LEGAL1
Total time: 18.05438670000087s
Average time per run: 1.805438670000087e-07ms
Median Time taken: 0.00013080000000087466s
LEGAL2
Total time: 13.838225100003264s
Average time per run: 1.3838225100003265e-07ms
Median Time taken: 0.00011419999999873198s
LEGAL3
Total time: 10.26031260000149s
Average time per run: 1.026031260000149e-07ms
Median Time taken: 9.540000000640703e-05s


## _

In [11]:
'''
Utility Imports, Functions and Classes
'''

from dtype import Move, Board

'''
Test Functions
'''



'''
Testing Hyperparameters
'''

functions = [dummy]    # the first function to run seems to have inflated time to run
n_iter = 100000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data

    # loop through all functions
    for func_name, func in func_dict.items():

        # time the run
        start = ttt()
        func()
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 100000


## _

In [ ]:
'''
Utility Imports, Functions and Classes
'''

from dtype import Move, Board

'''
Test Functions
'''



'''
Testing Hyperparameters
'''

functions = [dummy]    # the first function to run seems to have inflated time to run
n_iter = 100000

total_runs = n_iter

time_log = {}
func_dict = dict([(func.__name__, func) for func in functions])


'''
Run Speed Test
'''

for _ in range(n_iter):
    # generate random data

    # loop through all functions
    for func_name, func in func_dict.items():

        # time the run
        start = ttt()
        func()
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''
Display Statistics of Speed Test
'''

display_stats(time_log, total_runs)

Number of runs for all functions: 100000
